# Proyecto Capstone - La Batalla de los Vecindarios
#### Se busca mejorar la inversión en el ámbito inmobiliario en Argentina, para esto queremos encontrar las provincias que han tenido mayor auge en este sector. Contamos con 2 bases de datos proporcionadas por datos.gob.ar. La primera nos da informacion de los ultimos desarrollos realizados en Argetina y la segunda son las 24 pronvincias que tiene el país.

In [1]:
import numpy as np  # librería para manejar datos vectorizados
import pandas as pd # librería para procesar datos como dataframes 


In [2]:
import matplotlib.pyplot as plt #librería para graficar
%matplotlib inline 

In [3]:
# librería para manejar archivos JSON 
import json 

#!conda install -c conda-forge geopy --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare 
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud

import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas

# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 

print('Libraries imported.')

Libraries imported.


In [5]:
desarrollos=pd.read_csv("https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.29/download/localidades-censales.csv")
provincias=pd.read_csv("https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.7/download/provincias.csv")
desarrollos

,categoria,centroide_lat,centroide_lon,departamento_id,departamento_nombre,fuente,funcion,id,municipio_id,municipio_nombre,nombre,provincia_id,provincia_nombre
0,Localidad simple,-37.823387,-60.484640,6014.0,Adolfo Gonzales Chaves,INDEC,NaN,6014030,60014.0,Adolfo Gonzales Chaves,Juan E. Barra,6,Buenos Aires
1,Localidad simple,-37.625498,-58.746186,6063.0,Balcarce,INDEC,NaN,6063030,60063.0,Balcarce,Napaleofú,6,Buenos Aires
2,Localidad simple,-37.542635,-58.340719,6063.0,Balcarce,INDEC,NaN,6063040,60063.0,Balcarce,Ramos Otero,6,Buenos Aires
3,Localidad simple,-38.012284,-58.355378,6063.0,Balcarce,INDEC,NaN,6063050,60063.0,Balcarce,San Agustín,6,Buenos Aires
4,Localidad simple,-37.859629,-57.980601,6063.0,Balcarce,INDEC,NaN,6063060,60063.0,Balcarce,Villa Laguna La Brava,6,Buenos Aires
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3521,Componente de localidad compuesta,-26.815571,-65.269158,90119.0,Yerba Buena,INDEC,CABECERA_DEPARTAMENTO,90119030,908560.0,Municipalidad de Yerba Buena,Yerba Buena - Marcos Paz,90,Tucumán
3522,Localidad simple,-54.637686,-67.766941,94015.0,Ushuaia,INDEC,NaN,94015010,NaN,NaN,Laguna Escondida,94,"Tierra del Fuego, Antártida e Islas del Atlánt..."
3523,Localidad simple,-54.803640,-68.316062,94015.0,Ushuaia,INDEC,CAPITAL_PROVINCIA,94015020,940014.0,Ushuaia,Ushuaia,94,"Tierra del Fuego, Antártida e Islas del Atlánt..."
3524,Localidad simple,-53.787047,-67.713235,94008.0,Río Grande,INDEC,CABECERA_DEPARTAMENTO,94008010,940007.0,Río Grande,Río Grande,94,"Tierra del Fuego, Antártida e Islas del Atlánt..."


In [6]:
provincias

,categoria,centroide_lat,centroide_lon,fuente,id,iso_id,iso_nombre,nombre,nombre_completo
0,Provincia,-26.875397,-54.651697,IGN,54,AR-N,Misiones,Misiones,Provincia de Misiones
1,Provincia,-33.757726,-66.028130,IGN,74,AR-D,San Luis,San Luis,Provincia de San Luis
2,Provincia,-30.865368,-68.889491,IGN,70,AR-J,San Juan,San Juan,Provincia de San Juan
3,Provincia,-32.058874,-59.201448,IGN,30,AR-E,Entre Ríos,Entre Ríos,Provincia de Entre Ríos
4,Provincia,-48.815485,-69.955762,IGN,78,AR-Z,Santa Cruz,Santa Cruz,Provincia de Santa Cruz
5,Provincia,-40.405796,-67.229330,IGN,62,AR-R,Río Negro,Río Negro,Provincia de Río Negro
6,Provincia,-43.788623,-68.526759,IGN,26,AR-U,Chubut,Chubut,Provincia del Chubut
7,Provincia,-32.142933,-63.801753,IGN,14,AR-X,Córdoba,Córdoba,Provincia de Córdoba
8,Provincia,-34.629887,-68.583123,IGN,50,AR-M,Mendoza,Mendoza,Provincia de Mendoza
9,Provincia,-29.685776,-67.181736,IGN,46,AR-F,La Rioja,La Rioja,Provincia de La Rioja


In [7]:
regiones=pd.DataFrame(desarrollos.groupby(["provincia_nombre"]).size(),columns=['num_desarrollos']).sort_values(by="num_desarrollos")
regiones["Provincias"]=regiones.index
regiones

,num_desarrollos,Provincias
provincia_nombre,,
Ciudad Autónoma de Buenos Aires,1,Ciudad Autónoma de Buenos Aires
"Tierra del Fuego, Antártida e Islas del Atlántico Sur",4,"Tierra del Fuego, Antártida e Islas del Atlánt..."
Santa Cruz,27,Santa Cruz
Neuquén,60,Neuquén
Formosa,62,Formosa
Chubut,69,Chubut
La Rioja,73,La Rioja
Corrientes,77,Corrientes
San Juan,82,San Juan


In [8]:
geo_prov=regiones.merge(provincias,left_on="Provincias",right_on="nombre")
geo_prov.sort_values(by="num_desarrollos")

,num_desarrollos,Provincias,categoria,centroide_lat,centroide_lon,fuente,id,iso_id,iso_nombre,nombre,nombre_completo
0,1,Ciudad Autónoma de Buenos Aires,Ciudad Autónoma,-34.614493,-58.445856,IGN,2,AR-C,Ciudad Autónoma de Buenos Aires,Ciudad Autónoma de Buenos Aires,Ciudad Autónoma de Buenos Aires
1,4,"Tierra del Fuego, Antártida e Islas del Atlánt...",Provincia,-82.521518,-50.742749,IGN,94,AR-V,Tierra del Fuego,"Tierra del Fuego, Antártida e Islas del Atlánt...","Provincia de Tierra del Fuego, Antártida e Isl..."
2,27,Santa Cruz,Provincia,-48.815485,-69.955762,IGN,78,AR-Z,Santa Cruz,Santa Cruz,Provincia de Santa Cruz
3,60,Neuquén,Provincia,-38.641758,-70.118571,IGN,58,AR-Q,Neuquén,Neuquén,Provincia del Neuquén
4,62,Formosa,Provincia,-24.894973,-59.932441,IGN,34,AR-P,Formosa,Formosa,Provincia de Formosa
5,69,Chubut,Provincia,-43.788623,-68.526759,IGN,26,AR-U,Chubut,Chubut,Provincia del Chubut
6,73,La Rioja,Provincia,-29.685776,-67.181736,IGN,46,AR-F,La Rioja,La Rioja,Provincia de La Rioja
7,77,Corrientes,Provincia,-28.774305,-57.801219,IGN,18,AR-W,Corrientes,Corrientes,Provincia de Corrientes
8,82,San Juan,Provincia,-30.865368,-68.889491,IGN,70,AR-J,San Juan,San Juan,Provincia de San Juan
9,85,San Luis,Provincia,-33.757726,-66.028130,IGN,74,AR-D,San Luis,San Luis,Provincia de San Luis


In [9]:
mayor_municipio=geo_prov.loc[[19,20,21,22,23]]
mayor_municipio=mayor_municipio.drop(["categoria","nombre","fuente","id","iso_id","nombre_completo","Provincias"],axis=1)
mayor_municipio.reset_index(drop=True,inplace=True)
mayor_municipio.rename(columns={"iso_nombre": "Provincia", "B": "c"},inplace=True)

In [10]:
address = 'Argentina'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Argentina are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Argentina are -34.9964963, -64.9672817.


In [12]:
map_costa = folium.Map(location=[latitude, longitude], zoom_start=4)

# añadir los marcadores al mapa
for lat, lng, label in zip(mayor_municipio['centroide_lat'], mayor_municipio['centroide_lon'], mayor_municipio['Provincia']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_costa)  
    
map_costa